In [ ]:
import subprocess
import sys
import boto3
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

# Retrieve the Jupyter Notebook token
jupyter_token = sys.argv[1] if len(sys.argv) > 1 else ""

# Retrieve the AWS region
session = boto3.session.Session()
current_region = session.region_name

# Retrieve the AWS account ID from the IAM user
sts_client = session.client('sts')
response = sts_client.get_caller_identity()
account_id = response['Account']

# Execute the Docker login command
docker_login_command = f"aws ecr get-login-password --region {current_region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{current_region}.amazonaws.com"
subprocess.run(["bash", "-c", docker_login_command], input=jupyter_token.encode("utf-8"))

# Build the Docker container
docker_build_command = "docker build -t medical-mixtral-7b-v250k ."
subprocess.run(["bash", "-c", docker_build_command])

# Create a new ECR repository
ecr_repository_name = "aimedical"
create_ecr_repository_command = f"aws ecr create-repository --repository-name {ecr_repository_name} --region {current_region}"
subprocess.run(["bash", "-c", create_ecr_repository_command])

# Tag the Docker image
tag_docker_image_command = f"docker tag medical-mixtral-7b-v250k:latest {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", tag_docker_image_command])

# Push the Docker image to ECR
push_docker_image_command = f"docker push {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", push_docker_image_command])

# Get the default SageMaker execution role
role = sagemaker.get_execution_role()
print("SageMaker Execution Role ARN:", role)

# Set up the ScriptProcessor to run the Jupyter notebook
script_processor = ScriptProcessor(command=['jupyter', 'nbconvert', '--to', 'notebook', '--execute', 'FineTune.ipynb'],
                                   image_uri=f'{account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest',
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.m5.xlarge')

# Run the script
script_processor.run()
